In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from astropy.coordinates import SkyCoord, AltAz, EarthLocation
from astropy.time import Time
from astropy import units as u

In [2]:
file = "nebulaedata.csv"

In [3]:
data = pd.read_csv(file)
data.rename(columns={'# name': 'name'}, inplace=True)

In [4]:
def conv_ra(rastr):
    
    h, m = map(float, rastr.split())
    return 15*h + 0.25*m

data["R.A."] = data["R.A."].apply(conv_ra)

In [5]:
def conv_dec(decstr):
    
    deg, m = map(float, decstr.split())
    if deg < 0: m *= -1
    return deg + m / 60

data["Dec. "] = data["Dec. "].apply(conv_dec)

In [6]:
at_stony = EarthLocation(lat=40.914224*u.deg, lon=-73.11623*u.deg, height=0)
skycoords = SkyCoord(data['R.A.']*u.deg, data['Dec. ']*u.deg, frame='icrs')

timestrs = ["2019-10-04T02:00:00", "2019-10-18T02:00:00", "2019-10-22T02:00:00"]
visible = pd.Series([False] * len(data))

for timestr in timestrs:
    
    times = Time([timestr] * len(data), format='isot', scale='utc')
    altaz = AltAz(location=at_stony, obstime=times)
    aacoords = skycoords.transform_to(altaz)
    data["alt-" + timestr[8:10]] = aacoords.alt
    visible |= aacoords.alt > 40*u.deg
    
data = data[visible]

In [7]:
data.sort_values('Mag.')

,Name,R.A.,Dec.,Mag.,"Size("")",alt-04,alt-18,alt-22
29,M 27,299.900,22.716667,7.3,340,59.482012,49.704097,46.786234
22,NGC 6543,269.650,66.633333,8.3,22,48.795135,43.335066,41.782174
39,NGC 7662,351.475,42.550000,8.6,17,73.288631,83.388371,86.142306
28,NGC 6826,296.200,50.516667,8.8,27,65.268586,56.538796,54.038453
36,NGC 7027,316.775,42.233333,9.6,18,80.633420,70.440405,67.539532
30,NGC 6891,303.800,12.700000,10.5,15,54.179767,45.759410,43.084927
0,NGC 40,3.250,72.533333,10.7,60,54.254188,56.765083,57.280236
2,M 76,25.575,51.566667,11.0,107,50.553069,59.093854,61.529541
37,NGC 7048,318.725,46.300000,11.0,60,80.675815,71.699689,68.998852
5,NGC 1501,61.750,60.916667,12.0,56,33.682360,39.976251,41.840295


In [21]:
Surface_Brightness = data['Mag.'] + 2.5*np.log10((data['Size(")'])**2)

In [22]:
Surface_Brightness.sort_values()

39    14.752245
22    15.012113
36    15.876363
28    15.956819
30    16.380456
3     17.569717
35    18.989700
0     19.590756
38    19.612113
37    19.890756
29    19.957395
32    20.217263
5     20.740940
2     21.146919
26    21.396703
33    21.672492
31    22.617263
dtype: float64